### Importar librerias

In [2]:
import os
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import storage

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

/Users/nicolascalarco/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Configuración de la conexión con GCP

#### Crear un cliente de BigQuery

In [3]:

# Configurar la variable de entorno
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/nicolascalarco/.ssh/analitica1.json"

# Leer las credenciales de la variable de entorno
credentials = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

# Crear el cliente de BigQuery con las credenciales
client = bigquery.Client(project='analitica-1-383323', credentials=credentials)

In [4]:
#Creo funcion para ejecutar las querys y guardar el resulado en un dataframe
def run_query(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

### Creación del modelo de Clasificación eComm
<p> Para esto se utilizo Bigquery ML como motor para entrengar y guardar los resultados. </p>

In [5]:
query_model_clasificador = """
CREATE OR REPLACE MODEL `analitica-1-383323.ecomm.users-automl1` 
options(
       MODEL_TYPE =  'AUTOML_CLASSIFIER' ,
       INPUT_LABEL_COLS = ['fuga'],
       BUDGET_HOURS = 3
 
) as 
SELECT 
antiguedad_meses,
CASE
when antiguedad_meses>=0 and antiguedad_meses<6 then 0
when antiguedad_meses>=6 and antiguedad_meses<(select avg(antiguedad_meses) from `analitica-1-383323.ecomm.users`) then 1
else 2
end as antiguedad_cliente,		
fuga	,		
ind_satisfaccion,		
case
when var_is<0 then -1
when var_is=0 then 0
when var_is>0 then 1
end as satisfaccion_stuacion,
nro_casos_soporte,		
var_nro_casos_soporte	,	
if(nro_casos_soporte=0 and var_nro_casos_soporte=0,0,1) as tuvo_problemas,
case
when var_nro_casos_soporte>0 then -1
when var_nro_casos_soporte=0 then 0
when var_nro_casos_soporte<0 then 1
end as soporte_situacion,	
prioridad_soporte	,
case
when prioridad_soporte>0 and prioridad_soporte<2 then 0
when prioridad_soporte>=2 and prioridad_soporte<3 then -1
when prioridad_soporte>=3 then -2
end as prioridad_soporte_situacion,
var_prioridad_soporte	,		
var_nro_login	,		
var_uso_servicio	,		
var_beneficio_usuario	,		
var_dias_ult_conexion	,
if(var_dias_ult_conexion>-1,-1,0) as conexion_ultimo_mes
FROM `analitica-1-383323.ecomm.users`"""

query_evaluacion_modelo_clasificador = """
SELECT * FROM ML.EVALUATE(MODEL `analitica-1-383323.ecomm.users-automl1`)
"""

In [6]:
#Ejecución de las querys
#run_query(query_model_clasificador)

#Guardo las metricas del modelo 
df_evaluacion_clasificacion =run_query(query_evaluacion_modelo_clasificador)

In [7]:
df_evaluacion_clasificacion

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.97491,0.97491,0.97491,0.97491,0.154337,0.990192


#### Explicación de los resultados

1. **Umbral (0.5000)**: El umbral es un valor límite que se utiliza para convertir las probabilidades de salida del modelo en clases. En este caso, el umbral es 0.5, lo que significa que si la probabilidad de que una observación pertenezca a una clase positiva es mayor o igual a 0.5, se clasifica como positiva, de lo contrario, se clasifica como negativa.

2. **Precisión (0.97491)**: La precisión mide la proporción de verdaderos positivos (VP) entre el total de positivos predichos (VP + falsos positivos [FP]). Un valor de 0.97491 indica que el 97.49% de las predicciones positivas del modelo son correctas.

3. **Recuperación (0.97491)**: También conocida como sensibilidad o tasa de verdaderos positivos, la recuperación mide la proporción de verdaderos positivos (VP) entre el total de casos positivos reales (VP + falsos negativos [FN]). Un valor de 0.9749 indica que el modelo identifica correctamente el 97.49% de los casos positivos.

4. **Exactitud (0.97491)**: La exactitud mide la proporción de predicciones correctas (VP + verdaderos negativos [VN]) entre el total de observaciones (VP + VN + FP + FN). Un valor de 0.9749 indica que el 97.49% de las predicciones del modelo son correctas.

5. **Puntuación F1 (0.97491)**: La puntuación F1 es la media armónica de la precisión y la recuperación, y se utiliza para evaluar el equilibrio entre estas dos métricas. Un valor de 0.9749 indica que el modelo tiene un buen equilibrio entre precisión y recuperación.

6. **Pérdida logística (0.1543)**: La pérdida logística es una función de pérdida utilizada en problemas de clasificación binaria. Mide la diferencia entre las probabilidades predichas y las etiquetas reales. Un valor más bajo indica un mejor rendimiento del modelo. En este caso, la pérdida logística es de 0.1543.

7. **Área bajo la ROC (0.9901)**: La curva ROC (Receiver Operating Characteristic) es una representación gráfica del rendimiento de un modelo de clasificación binaria en función de su sensibilidad y especificidad. El área bajo la curva (AUC) es una medida que cuantifica el rendimiento general del modelo. Un valor de 1 indica un modelo perfecto, mientras que un valor de 0.5 indica un modelo que no es mejor que una clasificación aleatoria. En este caso, el AUC es de 0.9901, lo que indica un excelente rendimiento del modelo.

##### Opinion personal

<p>Cuando se trata de conjuntos de datos desequilibrados, hay varias estrategias que se puedes utilizar para mejorar el rendimiento del modelo en la clasificación de casos de la clase minoritaria, por ejemplo:

1.Remuestreo (Resampling): Puedes utilizar técnicas de remuestreo para equilibrar las clases. Estas técnicas incluyen el sobremuestreo (oversampling) de la clase minoritaria, el submuestreo (undersampling) de la clase mayoritaria, o una combinación de ambos. La técnica de sobremuestreo SMOTE (Synthetic Minority Over-sampling Technique) es una opción popular para generar ejemplos sintéticos de la clase minoritaria.


2.Modificar el umbral de clasificación: Por defecto, muchos modelos de clasificación binaria utilizan un umbral de 0.5 para decidir si una observación pertenece a una clase u otra. Puedes ajustar este umbral para mejorar el rendimiento en la clase minoritaria. Por ejemplo, al disminuir el umbral, se podria aumentar el recall de la clase minoritaria a expensas de la precisión.

3.Utilizar métodos de ensamble ponderados: Los métodos de ensamble como Random Forest o XGBoost permiten asignar diferentes pesos a las clases durante el entrenamiento. Al asignar un mayor peso a la clase minoritaria, puedes hacer que el modelo sea más sensible a esa clase.

4.Entrenar modelos específicos de la clase minoritaria: Puedes entrenar un modelo específicamente para clasificar los casos de la clase minoritaria. Por ejemplo, se puede entrenar un modelo para detectar clientes que abandonan la suscripción y otro modelo para detectar aquellos que no lo hacen. Luego, se puede combinar las predicciones de ambos modelos para obtener una clasificación más precisa.

5.Agrega características adicionales: Si el modelo no puede capturar patrones en los datos, que se necesite agregar características adicionales (variables) al conjunto de datos. Asegurandose de que las características estén relacionadas con la variable objetivo y tengan poder predictivo.

6.Utilizar técnicas de aprendizaje profundo: Se puede considerar utilizar técnicas de aprendizaje profundo, como las redes neuronales, para abordar el problema del desequilibrio de clases. Estas técnicas pueden ser más efectivas en la captura de patrones complejos en los datos.</p>

### Creación del modelo de Clusterización eComm
<p> Para esto se utilizo Bigquery ML como motor para entrengar y guardar los resultados. </p>

In [16]:
query_model_kmeans = """
CREATE OR REPLACE MODEL `analitica-1-383323.ecomm.users-kmeans1` 
options(
       MODEL_TYPE =  'KMEANS' ,
       KMEANS_INIT_METHOD = 'KMEANS++',
       MAX_ITERATIONS = 50,
       EARLY_STOP = FALSE 
 
) as 
SELECT 
antiguedad_meses,
CASE
when antiguedad_meses>=0 and antiguedad_meses<6 then 0
when antiguedad_meses>=6 and antiguedad_meses<(select avg(antiguedad_meses) from `analitica-1-383323.ecomm.users`) then 1
else 2
end as antiguedad_cliente,				
ind_satisfaccion,		

case
when var_is<0 then -1
when var_is=0 then 0
when var_is>0 then 1
end as satisfaccion_stuacion,
nro_casos_soporte,		
var_nro_casos_soporte	,	
if(nro_casos_soporte=0 and var_nro_casos_soporte=0,0,1) as tuvo_problemas,
case
when var_nro_casos_soporte>0 then -1
when var_nro_casos_soporte=0 then 0
when var_nro_casos_soporte<0 then 1
end as soporte_situacion,	
prioridad_soporte	,
case
when prioridad_soporte>=0 and prioridad_soporte<2 then 0
when prioridad_soporte>=2 and prioridad_soporte<3 then -1
when prioridad_soporte>=3 then -2
end as prioridad_soporte_situacion,
var_prioridad_soporte	,		
var_nro_login	,		
var_uso_servicio	,		
var_beneficio_usuario	,		
var_dias_ult_conexion	,
if(var_dias_ult_conexion>-1,-1,0) as conexion_ultimo_mes
FROM `analitica-1-383323.ecomm.users`;
"""

query_evaluacion_modelo_kmeans = """
SELECT * FROM ML.EVALUATE(MODEL `analitica-1-383323.ecomm.users-kmeans1`)
"""

In [15]:
#Ejecución de las querys
run_query(query_model_kmeans)

#Guardo las metricas del modelo
df_evaluacion_kmeans =run_query(query_evaluacion_modelo_kmeans)


In [17]:
df_evaluacion_kmeans

,davies_bouldin_index,mean_squared_distance
0,1.904306,11.699775


#### Explicación de los resultados

1. **Índice Davies–Bouldin (1.9043)**: El índice Davies-Bouldin es una métrica de evaluación de la calidad de un agrupamiento (clustering) en un conjunto de datos. Evalúa la separación entre los clusters y la cohesión dentro de cada cluster. Un valor más bajo indica una mejor calidad del agrupamiento, ya que sugiere una mayor separación entre los clusters y una menor dispersión dentro de cada cluster. En este caso, el índice Davies-Bouldin es de 1.9043.

2. **Media de la distancia al cuadrado (11.6998)**: La media de la distancia al cuadrado es una medida que cuantifica la dispersión de los puntos en un espacio multidimensional. En el contexto de clustering, puede ser utilizado para evaluar qué tan dispersos están los puntos dentro de cada cluster. Un valor más bajo indica que los puntos están más cercanos al centroide de su respectivo cluster, lo que sugiere una mayor cohesión dentro del cluster. En este caso, la media de la distancia al cuadrado es de 11.6998.

### Armado de la logica para consultar predicciones del modelo

<p> Los modelos ya se encuentran entrenados y guardados en Bigquery ML, por lo que se debe crear una función que permita consultar las predicciones de los modelos. Una vez vez verificado se armara la canalización para la corrida automatica</p>

In [11]:
#armado insert de clientes: (Esto se incorporara se reemplazara por una api o servicio que procese los datos de los clientes)

query_universo_analisis_clientes = """
SELECT id,
antiguedad_meses,
antiguedad_cliente,
ind_satisfaccion,
satisfaccion_stuacion,
nro_casos_soporte,
var_nro_casos_soporte,
tuvo_problemas,
soporte_situacion,
prioridad_soporte,
prioridad_soporte_situacion,
var_prioridad_soporte,
var_nro_login,
var_uso_servicio,
var_beneficio_usuario,
var_dias_ult_conexion,
conexion_ultimo_mes
FROM `analitica-1-383323.ecomm.users_final` 
"""

df_universo_analisis_clientes = run_query(query_universo_analisis_clientes)
#Configurando el entorno de trabajo
project_id = 'analitica-1-383323'
dataset_name = 'ecomm'
table_name = 'users_universo_analisis'

# Especifica la tabla de destino en BigQuery
destination_table = f"{project_id}.{dataset_name}.{table_name}"

# Guarda el DataFrame en BigQuery, reemplazando la tabla existente (si existe)
df_universo_analisis_clientes.to_gbq(destination_table, project_id=project_id, if_exists='replace', credentials=credentials)


100%|██████████| 1/1 [00:00<00:00, 3542.49it/s]


In [12]:
query_prediccion_clasificacion = """
SELECT *  FROM ML.PREDICT(MODEL `analitica-1-383323.ecomm.users-automl`, (select *
 from `ecomm.users_universo_analisis`))
"""
df_prediccion_clasificacion = run_query(query_prediccion_clasificacion)
df_prediccion_clasificacion

,predicted_fuga,predicted_fuga_probs,id,antiguedad_meses,antiguedad_cliente,ind_satisfaccion,satisfaccion_stuacion,nro_casos_soporte,var_nro_casos_soporte,tuvo_problemas,soporte_situacion,prioridad_soporte,prioridad_soporte_situacion,var_prioridad_soporte,var_nro_login,var_uso_servicio,var_beneficio_usuario,var_dias_ult_conexion,conexion_ultimo_mes
0,0.0,"[{'label': 0.0, 'prob': 0.7940770387649536}, {...",18,98,2,170.0,0,0.0,0.0,0,0,0.000000,0,0.000000,2.0,0.0,0.0,-12.0,0
1,0.0,"[{'label': 0.0, 'prob': 0.7892003655433655}, {...",36,92,2,148.0,0,0.0,0.0,0,0,0.000000,0,0.000000,2.0,0.0,0.0,-3.0,0
2,0.0,"[{'label': 0.0, 'prob': 0.8034811019897461}, {...",72,84,2,170.0,0,0.0,0.0,0,0,0.000000,0,0.000000,50.0,0.0,0.0,-3.0,0
3,0.0,"[{'label': 0.0, 'prob': 0.7674580812454224}, {...",98,22,1,118.0,0,0.0,0.0,0,0,0.000000,0,0.000000,-4.0,0.0,-168.0,-3.0,0
4,0.0,"[{'label': 0.0, 'prob': 0.7706999182701111}, {...",116,28,1,240.0,0,0.0,0.0,0,0,0.000000,0,0.000000,-4.0,0.0,-132.0,-2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,0.0,"[{'label': 0.0, 'prob': 0.80570387840271}, {'l...",528,36,2,192.0,-1,2.0,2.0,1,-1,3.000000,-2,3.000000,-4.0,-8.0,54.0,1.0,-1
2008,0.0,"[{'label': 0.0, 'prob': 0.8331834077835083}, {...",1035,22,1,508.0,-1,12.0,4.0,1,-1,3.000000,-2,0.000000,94.0,-8.0,20862.0,0.0,-1
2009,0.0,"[{'label': 0.0, 'prob': 0.8091263175010681}, {...",812,54,2,324.0,-1,6.0,4.0,1,-1,2.666667,-1,0.666667,-52.0,2.0,-62.0,0.0,-1
2010,0.0,"[{'label': 0.0, 'prob': 0.7574629187583923}, {...",1081,14,1,288.0,-1,6.0,6.0,1,-1,2.666667,-1,2.666667,2.0,-6.0,-428.0,4.0,-1


In [19]:
query_prediccion_kmeans = """
SELECT *  FROM ML.PREDICT(MODEL `analitica-1-383323.ecomm.users-kmeans1`, (select *
 from `ecomm.users_universo_analisis`))
"""
df_prediccion_kmenas = run_query(query_prediccion_kmeans)
df_prediccion_kmenas

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,id,antiguedad_meses,antiguedad_cliente,ind_satisfaccion,satisfaccion_stuacion,nro_casos_soporte,var_nro_casos_soporte,tuvo_problemas,soporte_situacion,prioridad_soporte,prioridad_soporte_situacion,var_prioridad_soporte,var_nro_login,var_uso_servicio,var_beneficio_usuario,var_dias_ult_conexion,conexion_ultimo_mes
0,3,"[{'CENTROID_ID': 3, 'DISTANCE': 2.437554766764...",1824,4,0,0.0,0,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,-1
1,1,"[{'CENTROID_ID': 1, 'DISTANCE': 2.790516352365...",1538,12,1,206.0,1,0.0,-2.0,1,1,0.0,0,-3.0,20.0,2.0,160.0,-3.0,0
2,2,"[{'CENTROID_ID': 2, 'DISTANCE': 4.622004606616...",10,104,2,358.0,-1,4.0,2.0,1,-1,3.0,-2,0.0,20.0,-4.0,10.0,-1.0,0
3,1,"[{'CENTROID_ID': 1, 'DISTANCE': 2.350594521473...",995,20,1,320.0,1,0.0,-4.0,1,1,0.0,0,-3.0,28.0,4.0,-476.0,2.0,-1
4,2,"[{'CENTROID_ID': 2, 'DISTANCE': 4.226176271538...",872,50,2,390.0,-1,2.0,0.0,1,0,3.0,-2,0.0,58.0,-12.0,-76.0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,2,"[{'CENTROID_ID': 2, 'DISTANCE': 3.234738714018...",542,34,2,166.0,-1,4.0,2.0,1,-1,3.0,-2,0.0,-24.0,0.0,-302.0,-10.0,0
2008,3,"[{'CENTROID_ID': 3, 'DISTANCE': 3.261123223713...",29,76,2,154.0,-1,0.0,0.0,0,0,0.0,0,0.0,-6.0,0.0,1432.0,-1.0,0
2009,3,"[{'CENTROID_ID': 3, 'DISTANCE': 2.138571124580...",965,40,2,0.0,0,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,-6.0,31.0,-1
2010,3,"[{'CENTROID_ID': 3, 'DISTANCE': 2.478547310662...",1342,2,0,0.0,0,0.0,0.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,-1
